Recognizing Shapes in Objects with OpenCV
=========================================


Import libraries.

In [1]:
import cv2
import numpy as np
import os

In [2]:
# Print the current working directory
#print(os.getcwd())

Image filenames.

In [3]:
files = (
    'subway.jpg',
    'breakfast.jpg',
    'dinner.jpg',
    'building.jpg',
)
f =  files[0] # Assuming images are in the current working directory

Define a function for viewing images.

In [4]:
def view_image(i):
    cv2.imshow('view',i)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

Read an image from file.

In [5]:
i = cv2.imread(f)
view_image(i)

Inspect image contents

In [6]:
print(i.shape)
print(i[0,0,:])
    

(640, 427, 3)
[22 24  4]


Gray-scale

In [7]:
i_gray = cv2.cvtColor(i, cv2.COLOR_BGR2GRAY)
print(i_gray.shape)
print(i_gray[0,0])
view_image(i_gray)

(640, 427)
18


X gradient

In [8]:
sobelx = cv2.Sobel(i_gray,cv2.CV_64F, 1 ,0)
abs_sobelx = np.absolute(sobelx)
view_image(abs_sobelx / np.max(abs_sobelx))

Y gradient

In [9]:
sobely = cv2.Sobel(i_gray,cv2.CV_64F, 0 ,1)
abs_sobely = np.absolute(sobely)
view_image(abs_sobely / np.max(abs_sobely))

Magnitude of gradient vector

In [10]:
magnitude = np.sqrt(sobelx**2 + sobely**2)
view_image(magnitude/np.max(magnitude))

Canny edge detection

In [11]:
edges = cv2.Canny(i_gray,200,250)
view_image(edges)

Hough transform for lines

In [12]:
lines = cv2.HoughLinesP(
    edges,
    rho = 1, 
    theta = 1. * np.pi / 180.0,
    threshold = 20,
    minLineLength = 25,
    maxLineGap = 5
)

i_lines = i.copy()

if lines is not None:
    for l in lines:
        x1, y1, x2, y2 = l[0]
        cv2.line(i_lines, (x1, y1), (x2, y2), (0, 0, 255), thickness=3)
    view_image(i_lines)
else:
    print("No lines detected.")


Hough transform for circles

In [13]:
circles = cv2.HoughCircles(
    i_gray,
    method = cv2.HOUGH_GRADIENT,
    dp = 2,
    minDist = 35,
    param1 = 150,
    param2 = 40,
    minRadius = 15,
    maxRadius = 25
)

i_circles = i.copy()

if circles is not None:
    # Convert circles array to integer type
    circles = np.round(circles[0, :]).astype("int")
    
    for (x, y, r) in circles:
        cv2.circle(
            i_circles,
            (x, y),       # Center of the circle
            r,            # Radius of the circle
            (0, 0, 255),  # Color (BGR)
            thickness=3   # Thickness of the circle outline
        )
    view_image(i_circles)
else:
    print("No circles detected.")

Blur the image for optimizing circle detection

In [14]:
i_blurred = cv2.GaussianBlur(
    i_gray,
    ksize =(21,21),
    sigmaX=0,
)
view_image(i_blurred)


Circle detection on blurred image

In [15]:
# Detect circles on the blurred image
circles = cv2.HoughCircles(
    i_blurred,
    method = cv2.HOUGH_GRADIENT,
    dp = 2,
    minDist = 35,
    param1 = 150,
    param2 = 40,
    minRadius = 15,
    maxRadius = 25
)

i_circles = i.copy()

if circles is not None:
    circles = np.round(circles[0, :]).astype("int")
    
    for (x, y, r) in circles:
        cv2.circle(
            i_circles,
            (x, y),
            r,
            (0, 0, 255),
            thickness=3
        )
    view_image(i_circles)
else:
    print("No circles detected.")